In [20]:
import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

# Establishing a Connection
Running the following code will connect the notebook to the AMIDatabase:

In [2]:
%load_ext sql
%sql postgresql://localhost/amidatabase

'Connected: @amidatabase'

We can check the version of PostgreSQL that we are using:

In [2]:
pg_version=%sql select version()
print(pg_version)

 * postgresql://localhost/amidatabase
1 rows affected.
+------------------------------------------------------------+
|                          version                           |
+------------------------------------------------------------+
| PostgreSQL 11.2, compiled by Visual C++ build 1915, 64-bit |
+------------------------------------------------------------+


# Creating PostgreSQL Tables from Pandas DataFrame
We can convert Pandas DataFrames to PostgreSQL tables in our database with some help from sqlalchemy. We must first create an engine using the create_engine command"

In [29]:
engine = create_engine('postgresql+psycopg2://Blake Bullwinkel:postgres@localhost/amidatabase')

As an example, we create a simple dataframe and use to_sql to save it as a PostgreSQL table called 'test' in our database. Note that PostgreSQL commands must be preceded by "%sql"

In [18]:
raw_data = {'first_name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'], 
        'last_name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze'], 
        'age': [42, 52, 36, 24, 73], 
        'preTestScore': [4, 24, 31, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70]}
df = pd.DataFrame(raw_data, columns = ['first_name', 'last_name', 'age', 'preTestScore', 'postTestScore'])
%sql DROP TABLE IF EXISTS test
df.to_sql('test',engine)

 * postgresql://localhost/amidatabase
Done.


We can now run SQL queries on this table as normal!

In [19]:
%sql SELECT * FROM test LIMIT 4

 * postgresql://localhost/amidatabase
4 rows affected.


index,first_name,last_name,age,preTestScore,postTestScore
0,Jason,Miller,42,4,25
1,Molly,Jacobson,52,24,94
2,Tina,Ali,36,31,57
3,Jake,Milner,24,2,62


## Loading Blue Flight Path Data
In a different notebook, "Blue Long Path Shifting.ipynb" I corrected the Segment_code values for each of the blue flight paths. We first load each of these corrected files into a DataFrame and add two fields from the .txt file name that uniquely identify the run, EventStartTime and Participant_id. Then we use to_sql() to save each dataframe as a table in the database (depending on how many files we are reading in, this may take a while).

In [44]:
basepath = r'C:\Users\Blake Bullwinkel\Downloads\WSSCGQWVS Corrected Sim Segments\Blue Path Blake\Shifted Blue Long Paths'
with os.scandir(basepath) as entries:
    for entry in entries:
        # we read all files in the directory that end in .txt
        if entry.name[-4:]=='.txt':
            path = pd.read_csv(entry,sep='	')
            path['EventStartTime'] = entry.name[-23:-10]
            path['Participant_id'] = entry.name[-9:-4]
            path.to_sql(entry.name[:-4],engine)

When querying these table, keep in mind that they are large and may crash the notebook without a LIMIT statement.

In [54]:
%sql SELECT * FROM "Shifted_AFRLAgent_5611620001340_P8666" LIMIT 5

 * postgresql://localhost/amidatabase
5 rows affected.


index,5611629842450,Aircraft_Quality,Timestamp_nano,EntityID_code,Latitude_deg,Longitude_deg,Altitude_ft,Heading_deg,Pitch_deg,Roll_deg,TrueAirSpeed_kt,VerticalSpeed_fpm,AircraftPathDeviation_Quality,Timestamp_nano.1,EntityID_code.1,Latitude_deg.1,Longitude_deg.1,Altitude_m,Heading_deg.1,PreviousWayPoint_code,NextWayPoint_code,TargetWayPoint_code,DistanceToPath_m,DistanceToTargetWayPoint_m,TrafficEvent_Quality,Timestamp_nano.2,TrafficEventType_code,EntityID_code.2,Latitude_deg.2,Longitude_deg.2,Heading_deg.2,RelatedWayPoint_code,UAVDeploymentTimestamp_nano,NBackTask_Quality,Timestamp_nano.3,TaskUniqueID_code,TrialNumber,TimeAsked_nano,AnswerTimeUp_nano,TimeAnswered_nano,AnswerGiven_code,ExpectedAnswer_code,CurrentSymbol_code,TargetSymbol_code,ReactionTime_msec,IsTimeUp,WayPointCrossingEvent_Quality,Timestamp_nano.4,WayPointCrossingEvent_code,Latitude_deg.3,Longitude_deg.3,Heading_deg.3,PreviousWayPoint_code.1,NextWayPoint_code.1,TargetedWayPoint_code,Segment_code,NBackTaskTriggerEvent_Quality,Timestamp_nano.5,TaskUniqueID_code.1,TestCategory_code,EventOccurrenceAfterSeconds,CognitiveDemand_code,TriggerType_code,SimulationState_Quality,Timestamp_nano.6,RealTime,TotalTime,IsPaused,EventStartTime,Participant_id
0,5611633042780,1.0,88563091340100.0,0.0,31.4970226287842,-87.5434722900391,22355.990234375,12.6140413284302,-8.72556114196777,121.648651123047,649.583679199219,-8731.4794921875,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.0,88563091325600.0,12733.9248046875,688.2080078125,1.0,5611620001340,P8666
1,5611633042890,1.0,88563091360300.0,0.0,31.4970226287842,-87.5434722900391,22355.990234375,12.6140413284302,-8.72556114196777,121.648651123047,649.583679199219,-8731.4794921875,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5611620001340,P8666
2,5611633042950,1.0,88563091366600.0,0.0,31.4970226287842,-87.5434722900391,22355.990234375,12.6140413284302,-8.72556114196777,121.648651123047,649.583679199219,-8731.4794921875,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5611620001340,P8666
3,5611633042980,1.0,88563091372500.0,0.0,31.4970226287842,-87.5434722900391,22355.990234375,12.6140413284302,-8.72556114196777,121.648651123047,649.583679199219,-8731.4794921875,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5611620001340,P8666
4,5611668867620,1.0,88563448464100.0,0.0,31.4970226287842,-87.5434722900391,22355.990234375,12.6140413284302,-8.72556114196777,121.648651123047,649.583679199219,-8731.4794921875,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.0,88563448448100.0,12734.3154296875,688.2080078125,1.0,5611620001340,P8666


## Loading Gold Flight Data

In a different notebook, "Relabeled Gold Long Paths.ipynb" I corrected the waypoint labeling for each of the gold flight paths. We add each of these files to our database in the same way as above.

In [58]:
basepath = r'C:\Users\Blake Bullwinkel\Downloads\WSSCGQWVS Corrected Sim Segments\Gold Path Blake\Relabeled Gold Long Paths'
with os.scandir(basepath) as entries:
    for entry in entries:
        # we read all files in the directory that end in .txt
        if entry.name[-4:]=='.txt':
            path = pd.read_csv(entry,sep='	')
            path['EventStartTime'] = entry.name[-23:-10]
            path['Participant_id'] = entry.name[-9:-4]
            path.to_sql(entry.name[:-4],engine)

The command below displays all of the tables in our database. As expected, we have 15 tables from our 15 blue and gold flight path files.

In [59]:
%sql \dt

 * postgresql://localhost/amidatabase
15 rows affected.


Schema,Name,Type,Owner
public,Relabeled_AFRLAgent_6152807954238_P3061,table,Blake Bullwinkel
public,Relabeled_AFRLAgent_6168263773276_P5454,table,Blake Bullwinkel
public,Relabeled_AFRLAgent_6201001215825_P3130,table,Blake Bullwinkel
public,Relabeled_AFRLAgent_6206460016536_P7234,table,Blake Bullwinkel
public,Relabeled_AFRLAgent_6239379978497_P7438,table,Blake Bullwinkel
public,Relabeled_AFRLAgent_6366148086779_P5521,table,Blake Bullwinkel
public,Relabeled_AFRLAgent_6704895162782_P8641,table,Blake Bullwinkel
public,Shifted_AFRLAgent_5611620001340_P8666,table,Blake Bullwinkel
public,Shifted_AFRLAgent_6010571848831_P8099,table,Blake Bullwinkel
public,Shifted_AFRLAgent_6054129383014_P3233,table,Blake Bullwinkel
